In [3]:
%pip install -U FlagEmbedding
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 3.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 11.0 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.2.11-py3-none-any.whl size=171152 sha256=8c638b23b46b74fd2cec2119b02076d971b2762b8b026fa3dd00e09c6e62770b
  Stored in directory: /root/.cache/pip/wheels/70/2a/6e/89f48d04306d3981effeeab965819ea7fe29a7c94983d29024
Successfully built FlagEmbedding
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import pandas as pd
import numpy as np
import pickle
import json

from FlagEmbedding import FlagModel

In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")

Using device: cuda


In [7]:
# Load the dataset
corpus_path = '/kaggle/input/chunk-doc-512/chunk_doc_512.json'

# Load the corpus data
with open(corpus_path, 'r') as f:
    corpus = json.load(f)

In [8]:
chunk_ids = list(corpus.keys())
chunk_texts = list(corpus.values())

In [9]:
model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

----------using 2*GPUs----------


In [10]:
chunk_embedding = model.encode(chunk_texts)

Inference Embeddings:   0%|          | 0/3547 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Inference Embeddings: 100%|██████████| 3547/3547 [5:47:47<00:00,  5.88s/it]  


In [11]:
# save result to pickle
chunk_embedding_dict = {chunk_id: embedding for chunk_id, embedding in zip(chunk_ids, chunk_embedding)}
with open('/kaggle/working/chunk_embedding.pkl', 'wb') as f:
    pickle.dump(chunk_embedding_dict, f)